In [85]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from IPython.display import display
import io

def csch(x):
    """Compute the hyperbolic cosecant of x."""
    return 1 / np.sinh(x)

def _normalh(x, length, a, b, c, d):
    """Compute the normalized hyperbolic weight."""
    y = x - d
    if y == 0:
        return 1  # The center coefficient
    n = b * c
    return np.arcsinh((2 * b * y) / (length / a)) * csch(
        (2 * y) / ((length / a) * c)) / n

normalh = np.vectorize(_normalh)


def generate_filter_coefficients(length, a, b, c, d, window_type=None, beta=None):
    """Generate filter coefficients for the given length and parameters, optionally applying a window function."""
    if length % 2 == 0:
        length += 1
    mid = length // 2
    x = np.arange(-mid, mid + 1)
    weights = normalh(x, mid, a, b, c, d)
    sum_weights = np.sum(np.abs(weights))
    normalized_weights = weights / sum_weights

    # Apply window function if specified
    if window_type != "none":
        if window_type == 'hamming':
            window = np.hamming(length)
        elif window_type == 'blackman':
            window = np.blackman(length)
        elif window_type == 'kaiser':
            assert beta is not None, "Beta value must be provided for Kaiser window"
            window = np.kaiser(length, beta)
        else:
            raise ValueError("Unsupported window type")
        # Applying the window to the filter coefficients
        normalized_weights *= window

    return normalized_weights


def normh_ma(source, weights):
    """Apply a normalized hyperbolic moving average filter using generated coefficients."""
    
    # Apply the convolution with the normalized weights
    filtered = np.convolve(source, weights, mode='same')
    return filtered


def process_audio_file(input_file,
                       output_file,
                       weights):
    """Read an audio file, apply the filter, and write the output to a new file."""
    data, samplerate = sf.read(input_file)
    if data.ndim > 1:
        filtered_channels = []
        for channel in range(data.shape[1]):
            filtered_channel = normh_ma(data[:, channel],
                                        weights)
            filtered_channels.append(filtered_channel)
        filtered_data = np.column_stack(filtered_channels)
    else:
        filtered_data = normh_ma(data,
                                 weights)

    sf.write(output_file, filtered_data, samplerate)



def display_filter_responses(weights, samplerate, response_samples=2048):
    """Display the frequency and phase response of the filter."""
    w = np.fft.rfftfreq(response_samples, d=1 / samplerate)
    h = np.fft.rfft(weights, n=response_samples)
    amplitude_response = 20 * np.log10(
        np.abs(h) + np.finfo(float).eps)  # Adding epsilon to avoid log(0)
    phase_response = np.angle(h, deg=True)

    plt.figure(figsize=(12, 6))

    # Subplot for Frequency Response
    plt.subplot(2, 1, 1)
    plt.semilogx(w, amplitude_response)
    plt.title('Frequency Response')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Amplitude [dB]')
    plt.grid(True, which='both', axis='both')
    plt.ylim(bottom=-96, top=0)  # Setting y-axis limits from -96 dB to 0 dB
    # Formatting frequency labels to display in decimal format
    formatter = FuncFormatter(lambda x, _: f'{x:.0f}')
    plt.gca().xaxis.set_major_formatter(formatter)

    # Subplot for Phase Response
    plt.subplot(2, 1, 2)
    plt.semilogx(w, phase_response)
    plt.title('Phase Response')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Phase [Degrees]')
    plt.grid(True, which='both', axis='both')
    plt.gca().xaxis.set_major_formatter(
        formatter)  # Apply the same formatter as the amplitude plot

    plt.tight_layout()
    plt.show()



In [86]:
import io
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from ipywidgets import widgets, Output
from IPython.display import display, clear_output

window_options = ["none", "hamming", "blackman", "kaiser"]

widget_window_type = widgets.Dropdown(
    options=window_options,
    value="hamming",
    description="Window Type:",
    disabled=False,
)

filter_length = widgets.IntSlider(
    value=16,
    min=2,
    max=512,
    description="Filter Length:",
    tooltip=("Length of filter, as well as a in distribution. As value "
             "increases, change in concavity moves away from 0 and magnitude "
             "of concavity decreases."))

offset = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    description="Offset:",
    tooltip="Distribution mean:"
                "Changed the center of the distribution to an arbitrary point. "
                "Messes shtz up yo.")

beta = widgets.FloatSlider(
    value=5,
    min=0,
    max=20,
    description="Beta:",
    tooltip="Beta value for Kaiser window",
)

scale = widgets.FloatLogSlider(
    value=1,
    base=2,
    min=-10,
    max=10,
    description="Scale Factor:",
    tooltip=("Do be scalin yo"))

peak_height = widgets.FloatLogSlider(
    value=1,
    base=2,
    min=-10,
    max=10,
    description="Peak Height:",
    tooltip=("Steepness of curve of the distribution. Change of concavity "
             "approaches 0 and magnitude of concavity increases as value "
             "increases."))

tail_length = widgets.FloatLogSlider(
    value=1,
    base=2,
    min=-6,
    max=6,
    description="Tail Length:",
    tooltip=("Tail length value for the hyperbolic distribution. Largely "
             "decreases the magnitude of concavity as value increases, but is "
             "non-monotonic."))


output = Output()

# def update_widgets(change):
#     if widget_window_type.value == 'kaiser':
#         beta.layout.display = 'block'
#     else:
#         beta.layout.display = 'none'


def update_graph(*args):
    with output:
        clear_output(wait=True)
        weights = generate_filter_coefficients(filter_length.value,
                                               scale.value,
                                               peak_height.value,
                                               tail_length.value,
                                               offset.value * filter_length.value,
                                               widget_window_type.value,
                                               beta=beta.value)
        display_filter_responses(weights, 44100)


def do_processing(*args):
    with output:
        uploaded_file = input_file.value[-1]["content"]
        data, samplerate = sf.read(io.BytesIO(uploaded_file))
        weights = generate_filter_coefficients(filter_length.value,
                                               scale.value,
                                               peak_height.value,
                                               tail_length.value,
                                               offset.value * filter_length.value,
                                               widget_window_type.value,
                                               beta=beta.value)
        process_audio_file(input_file.value[-1]["name"], output_file.value, weights)


input_file = widgets.FileUpload(accept=".wav",
                                multiple=False,
                                description="Select an audio file",
                                tooltip="The audio file to be filtered")
output_file = widgets.Text(
    value="filtered_audio.wav",
    description="Output File:",
    disabled=False,
    tooltip=("The name of the output file, in the same directory as the "
             "input file"))
process_file = widgets.Button(description="Process Audio File")
# widget_window_type.observe(update_widgets, names='value')
filter_length.observe(update_graph, names='value')
offset.observe(update_graph, names='value')
beta.observe(update_graph, names='value')
peak_height.observe(update_graph, names='value')
tail_length.observe(update_graph, names='value')
widget_window_type.observe(update_graph, names='value')
scale.observe(update_graph, names='value')

# Display widgets
display(widget_window_type, beta, filter_length, offset, peak_height, tail_length, scale)
display(output)
display(input_file, output_file, process_file)
process_file.on_click(do_processing)


Dropdown(description='Window Type:', index=1, options=('none', 'hamming', 'blackman', 'kaiser'), value='hammin…

FloatSlider(value=5.0, description='Beta:', max=20.0, tooltip='Beta value for Kaiser window')

IntSlider(value=16, description='Filter Length:', max=512, min=2, tooltip='Length of filter, as well as a in d…

FloatSlider(value=0.0, description='Offset:', max=1.0, min=-1.0, tooltip='Distribution mean:Changed the center…

FloatLogSlider(value=1.0, base=2.0, description='Peak Height:', max=10.0, min=-10.0, tooltip='Steepness of cur…

...

FloatLogSlider(value=1.0, base=2.0, description='Scale Factor:', max=10.0, min=-10.0, tooltip='Do be scalin yo…

Output()

FileUpload(value=(), accept='.wav', description='Select an audio file', tooltip='The audio file to be filtered…

Text(value='filtered_audio.wav', description='Output File:', tooltip='The name of the output file, in the same…

Button(description='Process Audio File', style=ButtonStyle())